# Aula 1 - Algoritmos de Redução de Dimensionalidade - Análise de Componentes Principais

Na nossa primeira aula do novo módulo de Machine Learning, vamos continuar falando sobre algoritmos não-supervisionados, porém hoje teremos o foco em algoritmos de **redução de dimensionalidade**. Falaremos sobre os seguintes tópicos:

- Análise de Componentes Principais - PCA
- Aplicando PCA na análise exploratória de dados
- Aplicando PCA com o Pipeline
- Selecionando o número de componentes
- A matemática por trás do PCA

É comum que um problema de dados real possua muitas dimensões ou *features*, de modo que isso pode tornar o treinamento impraticável do ponto de vista de custo computacional além de tornar a solução tecnicamente difícil de ser encontrada. No entanto, conforme veremos ao final da aula, frequentemente é possível aplicar técnicas de redução dessa dimensionalidade, visto que nem sempre todas esses atributos são realmente importantes para a resposta.

Além disso, quanto mais dimensões o dataset possuir, maior será a quantidade de exemplos necessários para se generalizar um padrão existente. E isso cresce de maneira exponencial. Pensemos no seguinte caso: estamos interessados em prever se uma determinada ação vai subir ou descer, baseando-se em regras. Nossa análise assume que valores discretos são permitidos apenas para os atributos.

- **se o faturamento for maior que R$ 1 bi, a ação sobe...** - aqui analisamos uma única dimensão (faturamento) e portanto precisamos conhecer apenas o faturamento
- **mas, se a taxa de juros estiver muito alta, a ação pode descer ou subir, dependendo do faturamento...** - duas dimensões (faturamento, taxa de juros), sendo necessário, no mínimo, 4 casos de análise (faturamento > 1 bi, juros baixos; faturamento < 1 bi, juros baixos; faturamento > 1 bi; juros altos; faturamento < 1 bi, juros altos)
- **também importante considerar o volume de ações para determinar se sobe ou desce** - três dimensões, com 27 casos a serem analisados
- com 10 atributos, são necessários 1000 registros de situações para nosso sistema de regras entender o que acontece em cada caso e poder responder o que fazer.
- com 100 atributos, 1.000.0000 de registros. E assim por diante.

Isso ilustra para um exemplo simples, onde somente situações discretas de cada atributo existem. Imaginem como isso fica absurdamente impraticável com atributos contínuos que podem se combinar de diversas formas. Com muitas dimensões, o modelo de aprendizado pode não ter registros o suficiente para entender registros diferentes entre si, levando a um desempenho inadequado. Isso é conhecido como *Maldição da Dimensionalidade*, um fenômeno comum em conjuntos de dados de alta dimensionalidade. Felizmente, temos técnicas bem interessantes para lidar com esse tipo de situação.

## **1. Análise de Componentes Principais - PCA**

O termo PCA vem do inglês *Principal Component Analysis* e denomina uma técnica **não-supervisionada** que realiza uma transformação das variáveis originais num novo espaço de *features*, de modo que as novas componentes cartesianas (as *componentes principais*) sejam **ortogonais entre si** e representem a direção da **máxima variância** dos dados originais. Assim sendo:

> A primeira componente principal é um eixo cartesiano que maximiza a variância dos dados projetados em sua direção<br>A segunda componente principal é um outro eixo cartesiano, ortogonal ao primeiro, que também maximia a variância nesta direção.

<img src=https://austingwalters.com/wp-content/uploads/2014/11/gaussDist-labeled.png width=400>

Fatos importantes sobre os resultados da análise de componentes principais:

- Se o espaço original tem $n$ atributos, então será possível construir $n$ componentes principais
- Cada componente principal é uma **combinação linear dos atributos originais**. Ao contrário dos algoritmos de seleção de atributos que eliminam definitivamente as colunas do conjunto de dados, a análise de componentes principais (e outras de redução de dimensionalidade) promovem a combinação desses atributos em novos espaços
- As componentes principais são ortogonais entre si, 
- As componentes principais são linearmente independentes entre si, sendo que a PCA também pode ser utilizada para eliminar multicolinearidade antes da modelagem.

Com a aplicação da PCA, é espera-se que, no **espaço transformado das componentes principais**, a variância completa ou aproximadamente completa dos dados esteja contida nas $L$ primeiras componentes principais ($L < n$).

Em resumo, as principais aplicações da PCA podem ser elencadas:

- evitar a Maldição da Dimensionalidade
- Acelerar o treinamento de modelos, visto que os algoritmos têm menor quantidade de colunas para considerar
- Fornecer dados sem colinearidade
- Visualização de dados e análise exploratória de dados
- Eliminar ruído de medição

### **1.1. Interpretação das Componentes Principais**

Conforme dito, cada componente principal é uma **combinação linear dos atributos originais**. Assim sendo, o PCA pode ser interpretado como um **método que constrói um novo espaço de atributos** ou seja, um *feature map*. Isso é bem semelhante ao que é feito para os algoritmos de *Kernel SVM* que verificamos no módulo de ML2. No entanto, no caso do SVM, os novos espaços criados pela aplicação da *kernel function* eram sempre de dimensão superior ao dataset original. Em contrapartida, a PCA sempre produzirá um espaço com dimensão igual ou menor ao dataset original.

<img src=https://miro.medium.com/max/1200/1*V3JWBvxB92Uo116Bpxa3Tw.png width=700>

Sabendo que os novos atributos são ortogonais entre sim, elas maximizam a variância dos dados em cada sub-espaço de projeção.

Na prática, seja um espaço de input $\mathcal{X}$ de $n$ dimensões, tal que cada vetor de features $\vec{x} \in \mathcal{X}$ é dado por:

$ \vec{x} = \left ( x_1, x_2, x_3, \cdots , x_n \right) $

Após a aplicação do PCA, estaremos no espaço de componentes principais, tal que um vetor de features neste espaço, $\vec{x}_{PC}$, será dado por:

$ \vec{x}_{PC} = \left ( PC_1, PC_2, PC_3, \cdots , PC_n \right) $

Onde cada componente principal é uma combinação linear das features originais, isto é, 

$ PC_i = \sum_{k=1}^n \alpha_{i, k} x_k = \alpha_{i, 1} x_1 +  \alpha_{i, 2} x_2 + \cdots +  \alpha_{i, n} x_n$

Note que cada componente principal $i$ tem $n$ coeficientes diferentes $\alpha_{i, k}$, que acompanham cada uma das features $k$ do espaço original. Esse coeficientes $\alpha_{i,k}$ podem ser utilizados como forma de geradores de *insights* a respeito da estrutura dos dados, sendo comumente chamados de *efeitos* ou *pesos*. Vamos começar com um exemplo simples.

No caso do dataset iris, temos 4 features, de modo que o espaço original tem 4 dimensões, e um vetor de features é:

$ \vec{x} = (x_1, x_2, x_3, x_4)$.

Sendo que:

- $x_1$ : 'sepal_length', 
- $x_2$ : 'sepal_width', 
- $x_3$ : 'petal_length', 
- $x_4$ : 'petal_width'.

Assim, esperamos que após o PCA, o espaço de componentes principais também tenha 4 dimensões, isto é, 

$ \vec{x}_{PC} = \left ( PC_1, PC_2, PC_3, PC_4 \right) $

E as componentes principais serão: 

$ PC_1 = \alpha_{1, 1} x_1 + \alpha_{1, 2} x_2 + \alpha_{1, 3} x_3 + \alpha_{1, 4} x_4$

$ PC_2 = \alpha_{2, 1} x_1 + \alpha_{2, 2} x_2 + \alpha_{2, 3} x_3 + \alpha_{2, 4} x_4$

$ PC_3 = \alpha_{3, 1} x_1 + \alpha_{3, 2} x_2 + \alpha_{3, 3} x_3 + \alpha_{3, 4} x_4$

$ PC_4 = \alpha_{4, 1} x_1 + \alpha_{4, 2} x_2 + \alpha_{4, 3} x_3 + \alpha_{4, 4} x_4$

O algoritmo de PCA é bem sensível à ordem de grandeza dos atributos, visto que ele depende do cálculo das variâncias e covariâncias, como veremos posteriormente. Além disso, uma das hipóteses que ele faz a respeito dos dados é que eles estejam **centrados em relação à média**. Portanto, a padronização é a transformação indicada para a utilização do PCA.

Para a aplicação do algoritmo de PCA, utilizaremos o objeto `PCA`, do `scikit-learn`, cuja documentação pode ser encontrada [nesse link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

O atributo `components_` nos apresenta os coeficientes das combinações lineares $\alpha_{i,k}$ de cada componente principal e podem ser utilizados como uma forma de análise exploratória dos dados.

Ou seja,

$PC_1 = 0.527x_1 -0.253x_2 + 0.582x_3 + 0.566x_4$

$PC_2 = 0.348x_1 + 0.935x_2 + 0.027x_3 + 0.066x_4$

$PC_3 = -0.727x_1 + 0.222x_2 + 0.139x_3 + 0.634x_4$

$PC_4 = 0.269x_1 -0.114x_2 -0.801x_3 + 0.523x_4$

Construindo-se um gráfico de combinações em pares de componentes principais, podemos usar os coeficientes para determinar pontos parecidos e, assim, inferir a respeito da estrutura dos dados. Por exemplo, espécies com maiores 'sepal_lenght' tendem a serem agrupadas com valores maiores de $PC_1$, visto que o coeficiente associado é positivo. No entanto, maiores $PC_1$ tendem a possuir exemplares com menores 'sepal_width', visto que o coeficiente associado é negativo.

No entanto, é de se pensar de que ao mesmo tempo que reduzimos a dimesionalidade, perdemos em alguma outra coisa. Essa coisa é a informação. Quando paramos de olhar para todas as componentes principais, deixamos de extrair a informação intrínseca a essas componentes. Mesmo assim, essa informação "perdida" pode ser tão irrelevante, de forma que os resultados finais tem pouco ou nenhum impacto. Para entender como isso acontece, precisamos analisar qual a fração de variância explicada por cada componente.

Os gráficos mostram que, selecionando apenas as duas primeiras componentes principais, extraímos cerca de 95 % da informação original, porém reduzido a dimensionalidade original pela metade.

Esse tipo de análise é interessante para determinar o quanto de informação estaria sendo levada para uma possível modelagem futura. Nesse caso, estamos garantindo que modelando com 2 componentes principais, estamos retendo praticamente toda a informação inicial.

> Veja que a escolha de $L < n$ componentes principais pode ser vista como um procedimento de **feature selection**, mas feito **no espaço de componentes principais!**
> Como não são as features originais que são selecionadas, é comum nos referirmos a este procedimento como **redução de dimensionalidade** (e a dimensionalidade que é reduzida é do espaço de componentes principais)

De fato, note que **todas as 4 features originais** estão presentes em cada uma das PCs, como termos da combinação linear que define cada PC. E, naturalmente, todas as 4 estão presentes nas $L$ primeiras.

Mas como podemos aplicar a PCA em análise exploratória de dados?

## **2. PCA aplicada à análise exploratória de dados**

Um uso importante do algoritmo de PCA é na **interpretabilidade** dos dados de alta dimensionalidade. Nossa compreensão cognitiva nos limita a enxergar o mundo em 3 dimensões e, mesmo em 3 dimensões, pode ser difícil de enxergar estruturas claras dos dados. Sabendo que as componentes principais possuem todas os atributos originais em sua composição, podemos contruir gráficos bidimensionais para entender essas relações de forma mais simples.

Nota-se, neste exemplo, que a espécie 0 ('setosa') é linearmente separável das outras, possuindo baixos valores de $PC_1$. Pela análise anterior dos pesos da primeira componente, isso nos leva à conclusão de que essa espécie comumente possuem os menores valores de 'sepal_lenght', porém os maiores valores de 'sepal_width', em relação às outras espécies.

<hr>

**Exercício 1** Utilizando a base de dados `heart.csv`, vamos tentar desenvolver um diagnosticador simples a respeito do risco de um ataque cardíaco. Para isso, aplique o algoritmo de PCA de forma exploratória, nos seguintes passos:

- cheque a presença de valores ausentes
- selecione somente atributos numéricos
- realize o escalonamento com `StandardScaler`
- por meio de gráficos bidimensionais, dê uma sugestão do que pode ser usado para diagnosticar um alto risco de ataque cardíaco.

Segue o dicionário de dados, retirado da origem dos dados ([esse link](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?resource=download&select=heart.csv)):

- `age`: idade da pessoa
- `sex`: gênero
- `cp`: tipo de dor no peito (1 - angina típica, 2 - angina atípica, 3 - dor não angínica, 4 - sem dor)
- `exang` - se houve angina induzida por exercício (1 - sim, 0 - não)
- `trtbps` - batimentos por segundo em estado de repouso
- `chol` - concentração de colesterol
- `fbs` - concentração de açúcar no sangue > 120 mg/dl (1 - sim, 0 - não)
- `restecg` - resultados do eletrocardiograma em estado de repouso (0 - normal, 1 - anormalidades de onda do sinal; 2 - provável ou definitiva hipertrofia do ventrículo esquerdo)
- `thalachh` - máxima taxa de batimentos cardíacos alcançados
- `target` - 0 - menor chance de ataque cardíaco, 1 - maior chance

<hr>

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings

from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, r2_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, recall_score
from sklearn.metrics import accuracy_score
# ignorar warnings
warnings.filterwarnings('ignore')

In [24]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [25]:
df = pd.read_csv("heart.csv")

X = df.drop(columns="output")
y = df["output"]

# ====================

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=42,
                                                    stratify=y)
# ====================

lista_etapas = [("std_scaler", StandardScaler()), 
                ("knn", KNeighborsClassifier(n_neighbors=5))]

pipe = Pipeline(lista_etapas)

# ====================

grupo = KFold(n_splits=10, shuffle=True, random_state=42)

metrica = make_scorer(recall_score, pos_label="M")

results = cross_validate(estimator=pipe,
                         X=X_train, y=y_train,
                         cv=grupo,
                         scoring=metrica,
                         return_train_score=True)

df_results = pd.DataFrame(results)

In [26]:
df_results.describe()

,fit_time,score_time,test_score,train_score
count,10.000000,10.000000,0.0,0.0
mean,0.002406,0.002018,NaN,NaN
std,0.001003,0.000789,NaN,NaN
min,0.001638,0.001441,NaN,NaN
25%,0.001687,0.001464,NaN,NaN
50%,0.001907,0.001616,NaN,NaN
75%,0.002914,0.002271,NaN,NaN
max,0.004547,0.003662,NaN,NaN


## **3. PCA no Pipeline**

Conforme já mencionado, além de servir para reduzir a dimesionalidade, a PCA pode ser aplicada para gerar atributos independentes entre si, visto que alguns algoritmos assumem a independência. Assim sendo, podemos aplicar o PCA dentro dos pipelines de modelagem que já aprendemos.

Verifica-se que o modelo de regressão logística conseguiu compreender os padrões com o mesmo desempenho de classificação do que aquele treinado com todas os atributos. No caso do dataset utilizado, isso quase não fez diferença, mas é um fator que se deve levar em consideração quando trabalhamos com milhões de linhas e, em alguns casos, milhares de colunas.

Mas até o momento, estamos fazendo a seleção do número de componentes de forma bem manual. Vejamos quais são as possíveis formas de se realizar a seleção da quantidade de componentes principais a serem retidas.

## **4. Selecionando o Número de Componentes**

Existem alguns métodos listados em literatura que podem ajudar na seleção do número de componentes principais a serem retidas. Vejamos os mais conhecidos:

- Seleção manual - absoluta ou por variância relativa
- Validação cruzada
- Método do cotovelo
- Método da análise paralela

### **4.1. Seleção Manual**

A seleção manual é o que já temos feito até o momento. Se for absoluta, especifica-se quantas componentes são desejadas. Se for por variância relativa, especifica-se qual a fração mínima de variância original a ser retida.

Verificam-se que o total das variâncias explicadas ultrapassa o valor estipulado. Mas o algoritmo retém componentes até que o mínimo especificado seja alcançado ou ultrapassado.

### **4.2. Método do Cotovelo**

Bem semelhante ao realizado para os métodos de vizinhos mais próximos (kNN e KMeans), o método do cotovelo analisa quando a variância retida já não aumenta significativamente a variaância acumulada. Isso é matematicamente conhecido como *ponto de inflexão*.

### **4.3. Validação Cruzada**

Se o propósito for utilizar a PCA em modelagem, podemos tentar selecionar o número de componentes por meio de um algoritmo de busca com validação cruzada. Neste caso, não estamos muito interessados na interpretabilidade do modelo, mas sim em desempenho. O número de componentes é visto como um *hiperparâmetro* adicional a ser buscado.

Nesse caso, o melhor estimador considerou todas as componentes principais como importantes para melhorar o desempenho do modelo em relação aos modelos anteriores. Mas se a dimensionalidade é a mesma, qual seria vantagem de utilizar o PCA nesse caso? - *Ausência de multicolinearidade*

### **4.4. Método da Análise Paralela**

Trata-se de um método clássico reportado em literatura. Essencialmente, compara-se a variância retida no PCA aplicado ao dataset original com o PCA aplicado num dataset simulado, totalmente livre de multicolinearidade. O número de componentes a serem retidas é aquele cuja variância retida no dataset original é maior pela última vez que a variância retida no dataset simulado. Vejamos como isso acontece:

- Realizar uma cópia do dataset original
- Para cada coluna:
    - simular uma distribuição normal, com mesma média e desvio padrão da original
- aplicar a PCA no dataset original, com dimensão igual ao dataset
- aplicar a PCA no dataset simulado, com dimnesão igual ao dataset
- comparar a variância retida em cada um dos datasets

Nesse caso, o número de componentes a ser considerado, deveria ser apenas 1.

<hr>

**Exercício 2** Construa um modelo de classificação para o risco de ataque cardíaco. Requisitos:

- utilize Pipeline
- compare os desempenhos de classificação usando todos os métodos de seleção do número de componentes principais, com exceção da seleção manual.

<hr>

<hr>

**Exercício 3 [DESAFIO]** Utilizando o dataset `german_credit_data.csv`, realize um mini-projeto exploratório e de modelagem utilizando PCA. Para isso:

- verifique dados nulos
- selecione apenas variáveis numéricas
- utilize modelagem por Pipeline
- faça PCA para análise exploratória e tente produzir *insights* a respeito do que torna uma pessoa adimplente ou o contrário
- escalonamento por `StandardScaler`
- o modelo a ser escolhido é livre
- faça otimização de hiperparâmetros com `RandomizedSearchCV` e `StratifiedKFold`
- compare o desempenho do modelo classificado para todos os métodos de seleção de componentes principais aprendidos.

<hr>

## **5. A Matemática do PCA**

Considere a matriz de features $X_{N \times n}$

> $N$ linhas (observações), cada uma caracterizada por $n$ features no espaço original.

Cada observação $i$ é caracterizada pelo vetor de features $\vec{x}_i = (x_{i1}, x_{i2}, \cdots, x_{in}$), que são as linhas da matriz de features:

$$
  X = \left [ \begin{array}{ccccc}
x_{11} & x_{12} & x_{13} & \cdots & x_{1n}\\ 
x_{21} & x_{22} & x_{23} & \cdots & x_{2n}\\ 
\vdots & \vdots & \vdots & \cdots & \vdots  \\
x_{N1} & x_{N2} & x_{N3} & \cdots & x_{Nn}\\ 
   \end{array} \right ] 
$$

### Passo 1 - Escalonamento

Para dados em que as features originais $x_j$ estão em escalas diferentes, é necessário escalar os dados para que eles tenham média 0 e desvio padrão 1 (ou seja, usamos o `StandardScaler`). 

Isso porque os componentes são influenciados pela escala das variáveis, justamente porque as matrizes de covariâncias, $\Sigma$ ou $\hat{\Sigma} = S$, são sensíveis à escala de um par de variáveis. 

Considere:

- $\bar{x}_j$ a média da variável $x_j$; 
- $s(x_j)$ o desvio padrão de $x_j$; 

Sendo $i = 1, 2,3,4,\cdots, N$ e $j = 1, 2,3,4,\cdots, n$.

Com isso, a padronização pode ser realizada por meio da equação abaixo: 

- Média 0 e desvio padrão 1: 

$$ \tilde{x}_{ij}= \frac{x_{ij}-\bar{x_j}}{s(X_j)} $$ 

<br>

### Passo 2 - Cálculo da matriz de covariância

Calcular a matriz de **covariância**/**correlação**, que são dadas por:

$$
  S = \left [ \begin{array}{ccccc}
\hat{Var}(x_1) & \hat{Cov}(x_1x_2) & \hat{Cov}(x_1x_3) & \cdots & \hat{Cov}(x_1x_n)\\ 
\hat{Cov}(x_2x_1) &\hat{Var}(x_2)& \hat{Cov}(x_2x_3) & \cdots & \hat{Cov}(x_2x_n)\\ 
\vdots & \vdots & \vdots & \cdots & \vdots  \\
\hat{Cov}(x_nx_1) & \hat{Cov}(x_nx_2)  & \hat{Cov}(x_nx_3)  & \cdots & \hat{Var}(x_n)\\ 
   \end{array} \right ] 
$$

<br>
<br>

$$
  R = \left [ \begin{array}{ccccc}
1 & r(x_1x_2) & r(x_1x_3) & \cdots & r(x_1x_n)\\ 
r(x_2x_1) & 1 & r(x_2x_3) & \cdots & r(x_2x_n)\\ 
\vdots & \vdots & \vdots & \cdots & \vdots  \\
r(x_nx_1) & r(x_nx_2)  & r(x_nx_3)  & \cdots & 1\\ 
   \end{array} \right ] 
$$

Em que:

$$
 \begin{array}{ccc}
\hat{Var}(x_j) = \frac{\sum_{i=1}^{N}(x_{ij}-\bar{x}_j)}{N-1}, & 
\hat{Cov}(x_{j1},x_{j2}) = \frac{\sum_{i=1}^N(x_{ij1}-\bar{x_{j1}})(x_{ij2}-\bar{x_{j2}})}{N-1}, &
r(x_{j1},x_{j2}) = \frac{\hat{Cov}(x_{j1},x_{j2})}{S_{xj1}S_{xj2}}
   \end{array} 
$$

<br>

### Passo 3 - Determinação de autovalores e autovetores

As componentes principais são determinadas através da equação característica (equação de autovalores) da matriz S ou R:

$$det[R - \lambda I]= 0 $$

Em que $I$ é a matriz identidade de dimensão $n \times n $. 

Se R ou S tem posto completo igual a $n$, então $det[R - \lambda I]= 0$, que pode ser reescrito como $\mid R - \lambda I \mid = 0$, terá $n$ soluções. Lembrando que ter posto completo significa que nenhuma coluna é combinação linear de outra, ou seja, as colunas são ortogonais e linearmente independentes.

Considere que $\lambda_1,\lambda_2,\lambda_3, \cdots, \lambda_n$ sejam as raízes da equação característica de R ou S, então temos que  $\lambda_1 > \lambda_2 > \lambda_3 > \cdots, \lambda_n$. 

Chamamos $\lambda_i$ de **autovalores**. 

Além disso, para cada autovalor há um **autovetor** $\tilde{a}_i$ associado, $
  \tilde{a}_i = \left [ \begin{array}{c}
a_{i1}\\ 
a_{i2}\\ 
\vdots \\
a_{ip} \\ 
   \end{array} \right ] 
$

O cálculo do autovetor $\tilde{a}_i$, pode ser realizado considerando a seguinte propriedade:

$$ R\tilde{a}_i =  \lambda_i \tilde{a}_i $$

O autovetor deve ser normalizado, isso é,

$$ a_i = \frac{\tilde{a}_i }{\mid \tilde{a}_i  \mid}$$

Desta maneira, as componentes do vetor são tais que sua norma L2 é igual a 1.

<br>

### Passo 4 - Cálculo das componentes principais

O cálculo da i-ésima componente principal é dado por:

$$PC_i = a_{i1}x_1 + a_{i2}x_2 + a_{i3}x_3 + \cdots + a_{in}x_n $$

em que $a_{i1}$ são as componetes do autovetor $a_i$ associado ao autovalor $\lambda_i$.

Operacionalmente, o PCA se apoia em um procedimento matemático denominado **Singular Value Decomposition (SVD)**, que é uma forma de decompor qualquer matriz não quadrada $M_{m\times n}$,

$$M_{m\times n} = U_{m \times m}\Sigma_{m \times n}V_{n \times n}^{\dagger}$$

- onde $U$ é uma matriz unitária $m\times m$ real ou complexa;

- $\Sigma$ é uma matriz retangular diagonal $m\times n$ com números reais não-negativos na diagonal;

- e $V^{\dagger}$ (a conjugada transposta de $V$) é uma matriz unitária $n\times n$ real ou complexa. 

Os valores de $\Sigma$ são os chamados valores singulares de $M$. As $m$ colunas de $U$ e as $n$ colunas de $V$ são os chamados vetores singulares à esquerda e vetores singulares à direita de $A$, respetivamente.

<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Singular_value_decomposition_visualisation.svg/800px-Singular_value_decomposition_visualisation.svg.png width=300>